### 영화 리뷰에 따라 tag를 10개 정도 추출하는 모델 제작


In [10]:
#keybert 테스트
from keybert import KeyBERT

kw_model = KeyBERT()

reviews = [
    '너무 진지', 
    '무대 연출', 
    '진지해요 여자',
    '연출과 가창력', 
    '연주자의 소리', 
    '기반으로 연출', 
    '볼만합니다', 
    '피가로역 연주자', 
    '독특하구요 탄탄한'
    ]


full_text = " ".join(reviews)
keywords = kw_model.extract_keywords(full_text, top_n=30, keyphrase_ngram_range=(1, 2))

print([kw[0] for kw in keywords])


['가창력 연주자의', '연출 진지해요', '연출과 가창력', '연주자 독특하구요', '기반으로 연출', '여자 연출과', '피가로역 연주자', '연출 볼만합니다', '독특하구요 탄탄한', '볼만합니다 피가로역', '연주자의 소리', '진지해요 여자', '무대 연출', '연출과', '연주자의', '진지해요', '연출', '독특하구요', '연주자', '가창력', '너무 진지', '볼만합니다', '진지 무대', '피가로역', '소리 기반으로', '여자', '기반으로', '탄탄한', '진지', '소리']


In [13]:
keywords = kw_model.extract_keywords(full_text, top_n=7, keyphrase_ngram_range=(1, 1))


In [14]:
print([kw[0] for kw in keywords])

['연출과', '연주자의', '진지해요', '연출', '독특하구요', '연주자', '가창력']


In [7]:
import pandas as pd
from keybert import KeyBERT
import re

# 데이터 불러오기
df = pd.read_csv("1.megabox_reviews_merged_avg.csv")

# 리뷰 열 합치기
review_cols = ["리뷰1", "리뷰2", "리뷰3", "리뷰4", "리뷰5"]
df["전체리뷰"] = df[review_cols].fillna("").agg(" ".join, axis=1)

# KeyBERT 초기화
kw_model = KeyBERT()

# 키워드 후처리 함수
def clean_tags(tags):
    tags = list(set(tags))                          # 중복 제거
    tags = [t for t in tags if len(t) > 1]          # 너무 짧은 단어 제거
    tags = [re.sub(r"[이가을를은는에의]$", "", t) for t in tags]  # 조사 제거
    return tags

# 키워드 추출 함수 (3단계 추출)
def extract_keywords_third(text):
    try:
        # 1차 추출: 원본 리뷰에서 상위 30개 키워드
        first_keywords = kw_model.extract_keywords(text, top_n=30, keyphrase_ngram_range=(1, 2))
        first_tags = [kw[0] for kw in first_keywords]
        first_tags = clean_tags(first_tags)

        # 2차 추출: 첫 키워드 리스트를 다시 하나의 문장으로 만들어 입력
        keyword_text = " ".join(first_tags)
        second_keywords = kw_model.extract_keywords(keyword_text, top_n=10, keyphrase_ngram_range=(1, 2))
        second_tags = [kw[0] for kw in second_keywords]
        second_tags = clean_tags(second_tags)
        
        # 3차 추출: 2차 추출의 키워드를 다시 하나의 문장으로 만들어 입력
        third_keywords = kw_model.extract_keywords(" ".join(second_tags), top_n=5, keyphrase_ngram_range=(1, 2))
        third_tags = [kw[0] for kw in third_keywords]
        third_tags = clean_tags(third_tags)

        return ", ".join(third_tags)
    except:
        return ""

# 키워드 추출 적용
df["리뷰키워드"] = df["전체리뷰"].apply(extract_keywords_third)

# 결과 저장
df.to_csv("megabox_with_keywords.csv", index=False, encoding="utf-8-sig")
print("🎉 키워드 2단계 추출 완료 → megabox_with_keywords.csv 저장됨")


🎉 키워드 2단계 추출 완료 → megabox_with_keywords.csv 저장됨


In [2]:
# 위 단계가 너무 오래 걸리는 문제점 있어. Okt 쓰는 방향으로 변경
from keybert import KeyBERT
from konlpy.tag import Okt
import pandas as pd
import re

# 데이터 불러오기
df = pd.read_csv("1.megabox_reviews_merged_avg.csv")

# 리뷰 열 합치기
review_cols = ["리뷰1", "리뷰2", "리뷰3", "리뷰4", "리뷰5"]
df["전체리뷰"] = df[review_cols].fillna("").agg(" ".join, axis=1)

# 모델 초기화
kw_model = KeyBERT()
okt = Okt()

# 형태소 분석 기반 키워드 정제 함수
def extract_keywords_okt(text):
    try:
        keywords = kw_model.extract_keywords(text, top_n=10, keyphrase_ngram_range=(1, 2))
        raw_phrases = [kw[0] for kw in keywords]

        filtered_words = []
        for phrase in raw_phrases:
            tokens = okt.pos(phrase, stem=True)
            for word, tag in tokens:
                if tag in ["Noun", "Adjective", "Verb"] and len(word) > 1:
                    filtered_words.append(word)

        return ", ".join(sorted(set(filtered_words)))
    except:
        return ""

# 적용
df["형태소키워드"] = df["전체리뷰"].apply(extract_keywords_okt)
df.to_csv("megabox_with_keywords_okt.csv", index=False, encoding="utf-8-sig")


In [3]:
# 다른 도전...

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from keybert import KeyBERT

# 데이터 불러오기
df = pd.read_csv("1.megabox_reviews_merged_avg.csv")

# 리뷰 열 합치기
review_cols = ["리뷰1", "리뷰2", "리뷰3", "리뷰4", "리뷰5"]
df["전체리뷰"] = df[review_cols].fillna("").agg(" ".join, axis=1)

# KeyBERT 초기화
kw_model = KeyBERT()

# 1. KeyBERT로 리뷰별 상위 30개 키워드 추출
df["초기키워드"] = df["전체리뷰"].apply(
    lambda x: [kw[0] for kw in kw_model.extract_keywords(x, top_n=30, keyphrase_ngram_range=(1, 2))]
)

# 2. 키워드 리스트를 하나의 문자열로 결합 (TF-IDF 벡터화용)
df["초기키워드문장"] = df["초기키워드"].apply(lambda tags: " ".join(tags))

# 3. 전체 문서에 대해 TF-IDF 학습
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df["초기키워드문장"])
feature_names = vectorizer.get_feature_names_out()

# 4. 각 리뷰에서 TF-IDF 기준 상위 N개 키워드만 선택
def get_top_tfidf_keywords(row_idx, top_n=5):
    row = tfidf_matrix[row_idx]
    scores = row.toarray().flatten()
    top_indices = scores.argsort()[-top_n:][::-1]
    return ", ".join([feature_names[i] for i in top_indices])

df["TFIDF키워드"] = [get_top_tfidf_keywords(i, top_n=5) for i in range(len(df))]

# 결과 저장
df.to_csv("megabox_tfidf_filtered.csv", index=False, encoding="utf-8-sig")


In [4]:
# 또 다른 도전...

import pandas as pd
from keybert import KeyBERT
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer

# 1. 데이터 불러오기 및 리뷰 합치기
df = pd.read_csv("1.megabox_reviews_merged_avg.csv")
review_cols = ["리뷰1", "리뷰2", "리뷰3", "리뷰4", "리뷰5"]
df["전체리뷰"] = df[review_cols].fillna("").agg(" ".join, axis=1)

# 2. KeyBERT 및 Okt 초기화
kw_model = KeyBERT()
okt = Okt()

# 3. KeyBERT → Okt → 정제된 단어 리스트 추출
def extract_keywords_and_tokens(text):
    try:
        # KeyBERT 상위 10개 키워드
        keywords = kw_model.extract_keywords(text, top_n=10, keyphrase_ngram_range=(1, 2))
        phrases = [kw[0] for kw in keywords]

        # 형태소 분석: 명사, 형용사, 동사만 추출
        refined_words = []
        for phrase in phrases:
            tokens = okt.pos(phrase, stem=True)
            for word, tag in tokens:
                if tag in ["Noun", "Adjective", "Verb"] and len(word) > 1:
                    refined_words.append(word)

        return " ".join(refined_words)
    except:
        return ""

# 4. 리뷰별 정제된 단어 문장 생성
df["정제단어문장"] = df["전체리뷰"].apply(extract_keywords_and_tokens)

# 5. TF-IDF 학습
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df["정제단어문장"])
feature_names = vectorizer.get_feature_names_out()

# 6. TF-IDF 기준 상위 단어 추출 함수
def get_top_tfidf(row_idx, top_n=5):
    row = tfidf_matrix[row_idx]
    scores = row.toarray().flatten()
    top_indices = scores.argsort()[-top_n:][::-1]
    return ", ".join([feature_names[i] for i in top_indices])

# 7. 최종 키워드 생성
df["최종키워드"] = [get_top_tfidf(i) for i in range(len(df))]

# 8. 저장
df.to_csv("megabox_final_keywords_tfidf.csv", index=False, encoding="utf-8-sig")
print("🎉 최종 키워드 추출 및 저장 완료 → megabox_final_keywords_tfidf.csv")


🎉 최종 키워드 추출 및 저장 완료 → megabox_final_keywords_tfidf.csv
